### Visualize (Hand Coupling / Save Video)

In [4]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2R,r2quat,get_uv_dict_nc,get_p_target_nc
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig_psyonic.xml`

In [14]:
xml_path = '../asset/common_rig/scene_common_rig_hand.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=False)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [1,0.7,0.7,0.5]

hand_names = ['rthumb_l1','rthumb_l2','rthumb_l3','rthumb_end',
              'rindex_l0','rindex_l1','rindex_l2','rindex_l3','rindex_end',
              'rmiddle_l0','rmiddle_l1','rmiddle_l2','rmiddle_l3','rmiddle_end',
              'rring_l0','rring_l1','rring_l2','rring_l3','rring_end',
              'rpinky_l0','rpinky_l1','rpinky_l2','rpinky_l3','rpinky_end',
              'lthumb_l1','lthumb_l2','lthumb_l3','lthumb_end',
              'lindex_l0','lindex_l1','lindex_l2','lindex_l3','lindex_end',
              'lmiddle_l0','lmiddle_l1','lmiddle_l2','lmiddle_l3','lmiddle_end',
              'lring_l0','lring_l1','lring_l2','lring_l3','lring_end',
              'lpinky_l0','lpinky_l1','lpinky_l2','lpinky_l3','lpinky_end']
for hand_name in hand_names:
    hand_idx = env.body_names.index(hand_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==hand_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [1,0,0,0.7]
print ("Done.")

Done.


### Visualization (Hand Coupling / Save Video)

In [6]:
for (i,n) in enumerate (env.joint_names):
    print ("[%d] %s"%(i,n))

[0] base
[1] root1
[2] root2
[3] root3
[4] spine
[5] spinex
[6] spinez
[7] rc
[8] rs1
[9] rs2
[10] rs3
[11] re
[12] rw1
[13] rw2
[14] rw3
[15] rthumb_q1
[16] rthumb_q2
[17] rthumb_q3
[18] rindex_q1
[19] rindex_q2
[20] rindex_q3
[21] rmiddle_q1
[22] rmiddle_q2
[23] rmiddle_q3
[24] rring_q1
[25] rring_q2
[26] rring_q3
[27] rpinky_q1
[28] rpinky_q2
[29] rpinky_q3
[30] lc
[31] ls1
[32] ls2
[33] ls3
[34] le
[35] lw1
[36] lw2
[37] lw3
[38] lthumb_q1
[39] lthumb_q2
[40] lthumb_q3
[41] lindex_q1
[42] lindex_q2
[43] lindex_q3
[44] lmiddle_q1
[45] lmiddle_q2
[46] lmiddle_q3
[47] lring_q1
[48] lring_q2
[49] lring_q3
[50] lpinky_q1
[51] lpinky_q2
[52] lpinky_q3
[53] head1
[54] head2
[55] head3
[56] rp1
[57] rp2
[58] rp3
[59] rk
[60] ra1
[61] ra2
[62] ra3
[63] lp1
[64] lp2
[65] lp3
[66] lk
[67] la1
[68] la2
[69] la3


In [16]:
import pickle
import os
import cv2

HAND_COUPLING = False
SAVE_VIDEO = False

pkl_path = "../data/pkl/VAAI_DIRECT_15_02_a_M1.pkl"

with open(pkl_path,'rb') as f:
    data = pickle.load(f)

video = []
tick = 0
L = data['length']

q_org = np.load("../data/15a_inpaint.npy")
q_org[q_org==0] = data['qpos'][q_org==0]

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=300,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

while tick < L:
    q = q_org[200,:]
    p_root = data['root'][200,:]

    if HAND_COUPLING:
        q_couple = q
        
        coupled_joint_idx_list = [[22,23],[24,25,26],[27,28,29],[30,31,32],[33,34,35],
                                  [45,46],[47,48,49],[50,51,52],[53,54,55],[56,57,58]]

        coupled_joint_weights_list = [[1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],
                                      [1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2]]

        for i in range(len(coupled_joint_idx_list)):
            coupled_joint_idx = coupled_joint_idx_list[i]
            coupled_joint_weights = coupled_joint_weights_list[i]
            joint_sum = 0

            for j in range(len(coupled_joint_idx)):
                joint_sum += q[coupled_joint_idx[j]]

            joint_sum /= np.sum(coupled_joint_weights)

            for k in range(len(coupled_joint_idx)):
                q_couple[coupled_joint_idx[k]] = joint_sum*coupled_joint_weights[k]

        q = q_couple
    
    env.forward(q=q,INCREASE_TICK=True)
    env.set_p_root(root_name='base',p=p_root)

    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        env.render()  
        frame = env.grab_image()
        video.append(frame)

        for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
            axis_joint = env.model.jnt_axis[rev_joint_idx]
            p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
            axis_world = R_joint@axis_joint
            axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
            axis_len,axis_r = 0.02,0.002
            env.plot_arrow_fr2to(
                p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
                r=axis_r,rgba=axis_rgba)

    tick = tick + 1
    if tick == L:
        if not SAVE_VIDEO: tick = 0

env.close_viewer()

if SAVE_VIDEO:
    shape = video[0].shape
    if HAND_COUPLING: 
        video_folder = ("../video/") 
    else: 
        video_folder = ("../video/original/")
    if not os.path.isdir(video_folder): os.mkdir(video_folder)

    video_path = os.path.join(video_folder, "original" + pkl_path[24:26] + ".mp4")
    video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    fps = 50
    video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

    for i in range(len(video)):
        video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
        video_out.write(video_frame)
    video_out.release()

Pressed ESC
Quitting.


Exception: GLFW window does not exist but you tried to render.

### Repeat for all 80 sequences

In [ ]:
import numpy as np
import pickle
import cv2
import os

HAND_COUPLING = False
SAVE_VIDEO = True

data_types = ["a","b","c"]

for data_type in data_types:
    if data_type == "a":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "b":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "c":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    for data_number in data_numbers:
        data_name = "VAAI_DIRECT_" + data_number + "_" + data_type + "_M1"
        pkl_path = "../data/pkl/" + data_name + ".pkl"

        with open(pkl_path,'rb') as f:
            data = pickle.load(f)

        video = []
        tick = 0
        L = data['length']

        # Initialize MuJoCo viewer
        env.init_viewer(viewer_title='Common Rig H',viewer_width=1200,viewer_height=800,
                        viewer_hide_menus=True)
        env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,1.2])
        env.reset()

        while tick < L:
            q = data['qpos'][tick,:]
            p_root = data['root'][tick,:]

            if HAND_COUPLING:
                q_couple = q
                
                coupled_joint_idx_list = [[22,23],[24,25,26],[27,28,29],[30,31,32],[33,34,35],
                                  [45,46],[47,48,49],[50,51,52],[53,54,55],[56,57,58]]

                coupled_joint_weights_list = [[1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],
                                            [1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2]]

                for i in range(len(coupled_joint_idx_list)):
                    coupled_joint_idx = coupled_joint_idx_list[i]
                    coupled_joint_weights = coupled_joint_weights_list[i]
                    joint_sum = 0

                    for j in range(len(coupled_joint_idx)):
                        joint_sum += q[coupled_joint_idx[j]]

                    joint_sum /= np.sum(coupled_joint_weights)

                    for k in range(len(coupled_joint_idx)):
                        q_couple[coupled_joint_idx[k]] = joint_sum*coupled_joint_weights[k]

                q = q_couple
            
            env.forward(q=q,INCREASE_TICK=True)
            env.set_p_root(root_name='base',p=p_root)

            if env.loop_every(tick_every=1):
                # Plot world frame
                env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                        PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
                env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                        PLOT_AXIS=False,label="tick:[%d]"%(tick))
                env.render()  
                frame = env.grab_image()
                video.append(frame)

            tick = tick + 1

        env.close_viewer()

        if SAVE_VIDEO:
            shape = video[0].shape
            if HAND_COUPLING: 
                video_folder = ("../video/hand_coupled/") 
            else: 
                video_folder = ("../video/original/")
            if not os.path.isdir(video_folder): os.mkdir(video_folder)

            video_path = os.path.join(video_folder, pkl_path.split('/')[-1].split('.')[0] + ".mp4")
            video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
            fps = 50
            video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

            for i in range(len(video)):
                video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
                video_out.write(video_frame)
            video_out.release()

In [ ]:
data_types = ["a"]
sum_val = 0

for data_type in data_types:
    if data_type == "a":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "b":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "c":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    for data_number in data_numbers:
        data_name = "VAAI_DIRECT_" + data_number + "_" + data_type + "_M1"
        pkl_path = "../data/pkl/" + data_name + ".pkl"

        with open(pkl_path,'rb') as f:
            data = pickle.load(f)
        
        print(data_number[:2], sum(data['qpos'][:,13]))
        sum_val += abs(sum(data['qpos'][:,13]))

print(sum_val)

01 15.683210680106779
02 14.856558022435545
03 36.25531626061012
04 3.953381417513982
05 31.388818544724675
06 21.89539452849541
07 29.97112044110173
08 28.186125538225646
09 42.52237382138494
10 49.104104768769716
11 30.644001396200757
12 28.537766269845605
13 38.75032726019914
14 29.145928044719113
15 48.004678384570816
16 49.413781228854475
17 24.428352356091562
18 -107.87045837305723
19 29.38550303228959
20 34.147567541292176
21 44.05474990019921
22 17.60255765724679
23 75.32390469429119
24 26.555210927039813
25 19.482728175013957
26 20.34104099088271
27 13.900396866266773
911.4053571214292
